In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 2.csv
/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 4.csv
/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 5.csv


In [39]:
books_data = pd.read_csv('/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 5.csv')
books_data.head()

,Order Number,Order Status,Book Name,Order Date & Time,City,Payment Method,Total items,Total weight (grams)
0,307,Completed,Bitcoin Blockchain aur Crypto Currency - FREE ...,10/3/2019 15:03,faisalabad,NaN,2,200.0
1,308,Completed,Kaggle for Begginers,10/3/2019 15:54,Rawalpindi,Cash on Delivery (COD),1,200.0
2,309,Completed,Kaggle for Begginers/Bitcoin Blockchain aur Cr...,10/3/2019 16:09,Hyderabad,Cash on Delivery (COD),2,200.0
3,310,Cancelled,Kaggle for Begginers,10/3/2019 16:42,Rawalpindi,Cash on Delivery (COD),1,200.0
4,311,Completed,Bit Coin Block Chain aur Crypto Currency بٹ کو...,10/3/2019 16:44,Karachi,Cash on Delivery (COD),1,200.0


**Goals to Achieve**

* What is the best-selling book?
* Visualize order status frequency
* Find a correlation between date and time with order status
* Find a correlation between city and order status¶
* Find any hidden patterns that are counter-intuitive for a layman
* Can we predict number of orders, or book names in advance?

**Data Exploration**

In [40]:
books_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Order Number          19239 non-null  int64  
 1   Order Status          19239 non-null  object 
 2   Book Name             19237 non-null  object 
 3   Order Date & Time     19239 non-null  object 
 4   City                  19238 non-null  object 
 5   Payment Method        19229 non-null  object 
 6   Total items           19239 non-null  int64  
 7   Total weight (grams)  19239 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 1.2+ MB


In [41]:
books_data.describe()

,Order Number,Total items,Total weight (grams)
count,19239.000000,19239.000000,19239.000000
mean,52535.780238,6.051874,363.749675
std,15835.761148,576.938452,834.097325
min,307.000000,0.000000,200.000000
25%,48064.500000,1.000000,200.000000
50%,56436.000000,1.000000,200.000000
75%,63779.500000,1.000000,500.000000
max,71420.000000,80022.000000,55000.000000


Figuring out the maximum value in total items. Let's see what is in there.

In [42]:
books_data[books_data['Total items']== 80022]

,Order Number,Order Status,Book Name,Order Date & Time,City,Payment Method,Total items,Total weight (grams)
15981,66499,Returned,انٹرنیٹ سے پیسہ کمائیں,12/27/2020 1:37,City,EasyPaisa,80022,200.0


This entry doesn't seem valid. So, dropping it.

In [43]:
books_data=books_data.drop(15981) #index number 15981

In [44]:
books_data.describe()

,Order Number,Total items,Total weight (grams)
count,19238.000000,19238.000000,19238.000000
mean,52535.054424,1.892608,363.758187
std,15835.852717,5.754468,834.118169
min,307.000000,0.000000,200.000000
25%,48064.250000,1.000000,200.000000
50%,56435.500000,1.000000,200.000000
75%,63778.750000,1.000000,500.000000
max,71420.000000,440.000000,55000.000000


Now it looks good. Let's move on

Lets look at the uniqueness of data

In [45]:
books_data.nunique()

Order Number            19238
Order Status                3
Book Name                3525
Order Date & Time       18099
City                     4163
Payment Method              5
Total items                50
Total weight (grams)       63
dtype: int64

Let's break down the date into Day, Month and Year.
First of all we need to convert the particular column to Date and Time data type.

In [58]:
books_data['Order Date & Time']=pd.to_datetime(books_data['Order Date & Time'])

In [60]:
books_data['Year']=books_data['Order Date & Time'].dt.strftime('%Y')

In [61]:
books_data['Month']=books_data['Order Date & Time'].dt.strftime('%B')

In [62]:
books_data['Day']=books_data['Order Date & Time'].dt.strftime('%a')

In [63]:
books_data.head()

,Order Number,Order Status,Book Name,Order Date & Time,City,Payment Method,Total items,Total weight (grams),Year,Month,Day
0,307,Completed,Bitcoin Blockchain aur Crypto Currency - FREE ...,2019-10-03 15:03:00,faisalabad,COD,2,200.0,2019,October,Thu
1,308,Completed,Kaggle for Begginers,2019-10-03 15:54:00,Rawalpindi,COD,1,200.0,2019,October,Thu
2,309,Completed,Kaggle for Begginers/Bitcoin Blockchain aur Cr...,2019-10-03 16:09:00,Hyderabad,COD,2,200.0,2019,October,Thu
3,310,Cancelled,Kaggle for Begginers,2019-10-03 16:42:00,Rawalpindi,COD,1,200.0,2019,October,Thu
4,311,Completed,Bit Coin Block Chain aur Crypto Currency بٹ کو...,2019-10-03 16:44:00,Karachi,COD,1,200.0,2019,October,Thu


Now let's move on to cities.

In [64]:
books_data['City'].value_counts()

City
Karachi                                                                       2332
Lahore                                                                        1993
Islamabad                                                                      859
Rawalpindi                                                                     645
Faisalabad                                                                     478
                                                                              ... 
Distt.Rahim Yar Khan                                                             1
MAJID IQBAL SATTI PLOT # 150, SECTOR # 23 KORANGI INDUATRIAL AREA KARACHI.       1
Gulberg 3 Lahore                                                                 1
Sadat Pur Rehlein P /O Retra                                                     1
Mailsi, district Vehari                                                          1
Name: count, Length: 4163, dtype: int64

oohh.. There's too much clutter out there. To be on safer side, copying the current dataframe.

In [66]:
org_books_data = books_data.copy()

In [67]:
org_books_data.head()

,Order Number,Order Status,Book Name,Order Date & Time,City,Payment Method,Total items,Total weight (grams),Year,Month,Day
0,307,Completed,Bitcoin Blockchain aur Crypto Currency - FREE ...,2019-10-03 15:03:00,faisalabad,COD,2,200.0,2019,October,Thu
1,308,Completed,Kaggle for Begginers,2019-10-03 15:54:00,Rawalpindi,COD,1,200.0,2019,October,Thu
2,309,Completed,Kaggle for Begginers/Bitcoin Blockchain aur Cr...,2019-10-03 16:09:00,Hyderabad,COD,2,200.0,2019,October,Thu
3,310,Cancelled,Kaggle for Begginers,2019-10-03 16:42:00,Rawalpindi,COD,1,200.0,2019,October,Thu
4,311,Completed,Bit Coin Block Chain aur Crypto Currency بٹ کو...,2019-10-03 16:44:00,Karachi,COD,1,200.0,2019,October,Thu


In [49]:
books_data['Payment Method'].value_counts()

Payment Method
COD             17477
EasyPaisa         696
JazzCash          596
BankTransfer      459
Name: count, dtype: int64

There's consistency issue in data. Cash on delivery and Cash on Delivery (COD) are same. Let's make the data consistent. The above output is after replacing the values.

In [47]:
books_data['Payment Method']=books_data['Payment Method'].replace({'Cash on delivery':'COD','Cash on Delivery (COD)':'COD'})

There are some values missing in payment method. Let's assume that they are COD and fill those values.

In [51]:
books_data['Payment Method'].fillna('COD',inplace=True)

In [54]:
books_data['Payment Method'].unique()

array(['COD', 'EasyPaisa', 'JazzCash', 'BankTransfer'], dtype=object)